# Practica 1

## Introducción

En la actualidad, el almacenamiento de múltiples fuentes de información en un lago de datos se compone de diferentes opciones de almacenamiento para cada capa. Esto significa que se requiere realizar una preparación con los datos, a fin de que sean adecuados para su análisis o modelización posterior. En esta práctica realizaremos un tratamiento de datos con *Shell Script*,  donde inicialmente leeremos la información (extraer), luego filtraremos y depuraremos algunos datos (transformar), y finalmente los escribiremos en un nuevo archivo de carga (cargar). Estos procesos generalmente ocurren como pasos ETL (Extract, transform and load) en una canalización de datos entre capas, mediante procesamiento por lotes.\\

El conjunto de datos *titles.cvs* que trabajaremos, contiene los listados de todas las películas y programas de televisión disponibles en Netflix, junto con detalles como el año de realización, duración del contenido en minutos, géneros de producción, países participantes, etc.

## Lectura de datos

Las ejecuciones Bash desde Jupyter se han en local, ya que los resultados no son visibles en GitHub. A continuación asignamos el directorio de trabajo.

In [19]:
!cd ~/exercicis-linux/practiques/Practica1
!ls -l

total 1956
-rw-rw-r-- 1 patricia patricia   14597 oct 26 16:33 JupyterNotebookP1.ipynb
-rwxrw-r-- 1 patricia patricia     145 oct 26 16:23 MoviesP1.awk
-rw-rw-r-- 1 patricia patricia 1978945 oct 25 15:34 titles.cvs


En la lectura inicial de los datos, observamos que tenemos campos delimitados por comas (,) y otros por comillas ("):

In [20]:
!head titles.cvs

id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
ts300399,Five Came Back: The Reference Films,SHOW,"This collection includes 12 World War II-era propaganda films — many of which are graphic and offensive — discussed in the docuseries ""Five Came Back.""",1945,TV-MA,48,['documentation'],['US'],1.0,,,,0.6,
tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works as a night-time taxi driver in New York City where the perceived decadence and sleaze feed his urge for violent action  attempting to save a preadolescent prostitute in the process.,1976,R,113,['crime'  'drama'],['US'],,tt0075314,8.3,795222.0,27.612,8.2
tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur  accompanied by his squire  recruits his Knights of the Round Table  including Sir Bedevere the Wise  Sir Lancelot the Brave  Sir Robin the Not-Quite-So-Brave-As-Sir-Lancelot and Sir Galahad the Pure

Inicialmente, para no tener en cuenta las comas como separadores de campos, crearemos el archivo *MoviesP1.awk* que utilizaremos para analizar el fichero .cvs. En la sentencia BEGIN indicamos que el separador es por comas, y en la salida con NF (número de filas observamos si tenemos más filas que número de columnas (15).


BEGIN{
        FS=","
}

{    
        if(NF>15) print "NF = " NF
}


In [28]:
!cat titles.cvs | awk -f MoviesP1.awk

NF = 25
NF = 25
NF = 25
NF = 25
NF = 25
NF = 26
NF = 25
NF = 25
NF = 25
NF = 25
NF = 25
NF = 25
NF = 25
NF = 25


Este resultado nos indica que las separaciones por comas en algún campo de texto, también se está utilizando (como por ejemplo la descripción), por lo que reemplazamos mediante la función *gsub()* el texto correspondiente, y de ésta forma tenemos cada una de las filas igual que el número de columnas:

BEGIN{
        FS=","
}

{
        gsub(",,", ", ,")
        print "NF = " NF
}


In [30]:
!head titles.cvs | awk -f MoviesP1.awk

NF = 15
NF = 15
NF = 15
NF = 15
NF = 15
NF = 15
NF = 15
NF = 15
NF = 15
NF = 15


## Transformación de los Datos

1) Una vez hemos leído el fichero, un primer paso es limpiar los identificadores que no comiencen con los carácteres *tm* o *ts*, para lo cual nos ayudamos de la función index que nos devuelve el orden en que el caracter aparece por primera vez en una cadena:

{
        v1=$1
        if(index(v1, "tm")==1 || index(v1, "ts")==1)
        print  v1
}


In [33]:
!head titles.cvs | awk -f MoviesP1.awk

ts300399
tm84618
tm127384
tm70993
tm190788
ts22164
tm14873
tm185072
tm98978


2) Ahora eliminaremos los registros en los cuales el contenído del título no comience por un carácter alfanumèrico, para lo cual nos ayudamos de la siguiente expresión regular:

{
        v2=$2
        if(v2 !~/^[^A-Za-z]/)
        print v2  }


In [34]:
!head titles.cvs | awk -f MoviesP1.awk

title
Five Came Back: The Reference Films
Taxi Driver
Monty Python and the Holy Grail
Life of Brian
The Exorcist
Monty Python's Flying Circus
Dirty Harry
My Fair Lady
The Blue Lagoon


3) Ahora separaremos en dos ficheros diferentes, las películas de las series. Los campos que no tienen sentido, se guardan en un fichero a parte:

{ 
        v3=$3
        if(NR=1 || v3=="MOVIE") print > "file1.csv";
        if(NR=1 || v3=="SHOW") print > "file2.csv";
}


In [37]:
!head titles.cvs | awk -f MoviesP1.awk

In [40]:
!head file1.csv

id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
ts300399,Five Came Back: The Reference Films,SHOW,"This collection includes 12 World War II-era propaganda films — many of which are graphic and offensive — discussed in the docuseries ""Five Came Back.""",1945,TV-MA,48,['documentation'],['US'],1.0, ,, ,0.6,
tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works as a night-time taxi driver in New York City where the perceived decadence and sleaze feed his urge for violent action  attempting to save a preadolescent prostitute in the process.,1976,R,113,['crime'  'drama'],['US'], ,tt0075314,8.3,795222.0,27.612,8.2
tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur  accompanied by his squire  recruits his Knights of the Round Table  including Sir Bedevere the Wise  Sir Lancelot the Brave  Sir Robin the Not-Quite-So-Brave-As-Sir-Lancelot and Sir Galahad the P

In [41]:
!head file2.csv

id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
ts300399,Five Came Back: The Reference Films,SHOW,"This collection includes 12 World War II-era propaganda films — many of which are graphic and offensive — discussed in the docuseries ""Five Came Back.""",1945,TV-MA,48,['documentation'],['US'],1.0, ,, ,0.6,
tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works as a night-time taxi driver in New York City where the perceived decadence and sleaze feed his urge for violent action  attempting to save a preadolescent prostitute in the process.,1976,R,113,['crime'  'drama'],['US'], ,tt0075314,8.3,795222.0,27.612,8.2
tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur  accompanied by his squire  recruits his Knights of the Round Table  including Sir Bedevere the Wise  Sir Lancelot the Brave  Sir Robin the Not-Quite-So-Brave-As-Sir-Lancelot and Sir Galahad the P

## Proyecto GitHub

Los archivos del proyecto se encuentran publicados en https://github.com/RunCite/FI/tree/main/Practica1